Initial attemps at profiling had very confusing results; possibly because of module loading and i/o

Here, gypsy will be run and profiled on one plot, with no module loading/io recorded in profiling

# Characterize what is happening

In several places, we append data to a data frame

In [19]:
%%bash
grep --colour -nr append ../gypsy/*.py

../gypsy/GYPSYNonSpatial.py:1028:        densities_along_time.append({'N_bh_AwT': N_bh_AwT, 'N_bh_SwT': N_bh_SwT, 'N_bh_SbT': N_bh_SbT, 'N_bh_PlT': N_bh_PlT,
../gypsy/GYPSYNonSpatial.py:1163:            BA_Aw_DF = BA_Aw_DF.append({'BA_Aw':BA_AwB}, ignore_index=True)
../gypsy/GYPSYNonSpatial.py:1287:            BA_Sb_DF = BA_Sb_DF.append({'BA_Sb': BA_SbB}, ignore_index=True)
../gypsy/GYPSYNonSpatial.py:1422:            BA_Sw_DF = BA_Sw_DF.append({'BA_Sw': BA_SwB}, ignore_index=True)
../gypsy/GYPSYNonSpatial.py:1673:            BA_Pl_DF = BA_Pl_DF.append({'BA_Pl': BA_PlB}, ignore_index=True)
../gypsy/forward_simulation.py:367:            output_DF_Sw = output_DF_Sw.append({'BA_Sw':BA_SwT}, ignore_index=True)
../gypsy/forward_simulation.py:368:            output_DF_Sb = output_DF_Sb.append({'BA_Sb':BA_SbT}, ignore_index=True)
../gypsy/forward_simulation.py:369:            output_DF_Pl = output_DF_Pl.append({'BA_Pl':BA_PlT}, ignore_index=True)


Either in the way we do it, or by its nature, it is a slow operation.

In [17]:
import pandas as pd

In [23]:
help(pd.DataFrame.append)

Help on method append in module pandas.core.frame:

append(self, other, ignore_index=False, verify_integrity=False) unbound pandas.core.frame.DataFrame method
    Append rows of `other` to the end of this frame, returning a new
    object. Columns not in this frame are added as new columns.
    
    Parameters
    ----------
    other : DataFrame or Series/dict-like object, or list of these
        The data to append.
    ignore_index : boolean, default False
        If True, do not use the index labels.
    verify_integrity : boolean, default False
        If True, raise ValueError on creating index with duplicates.
    
    Returns
    -------
    appended : DataFrame
    
    Notes
    -----
    If a list of dict/series is passed and the keys are all contained in
    the DataFrame's index, the order of the columns in the resulting
    DataFrame will be unchanged.
    
    See also
    --------
    pandas.concat : General function to concatenate DataFrame, Series
        or Panel obj

There is nothing very clear about performance from the documentation. It may be worth examining the source, and of course googling append performance.

python - Improve Row Append Performance On Pandas DataFrames - Stack Overflow  
http://stackoverflow.com/questions/27929472/improve-row-append-performance-on-pandas-dataframes

python - Pandas: Why should appending to a dataframe of floats and ints be slower than if its full of NaN - Stack Overflow  
http://stackoverflow.com/questions/17141828/pandas-why-should-appending-to-a-dataframe-of-floats-and-ints-be-slower-than-if

python - Creating large Pandas DataFrames: preallocation vs append vs concat - Stack Overflow  
http://stackoverflow.com/questions/31690076/creating-large-pandas-dataframes-preallocation-vs-append-vs-concat

python - efficient appending to pandas dataframes - Stack Overflow  
http://stackoverflow.com/questions/32746248/efficient-appending-to-pandas-dataframes

python - Pandas append perfomance concat/append using "larger" DataFrames - Stack Overflow  
http://stackoverflow.com/questions/31860671/pandas-append-perfomance-concat-append-using-larger-dataframes

pandas.DataFrame.append — pandas 0.18.1 documentation  
http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html

# Decide on the action

Do not append in a loop. It makes a copy each time and the memory allocation is poor. Should have known; it's interesting to see it demonstrated in the wild!

Pre-allocate the dataframe length by giving it an index and assigning to the index

# MWE

In [29]:
%%timeit
d = pd.DataFrame(columns=['A'])
for i in xrange(1000):
    d.append({'A': i}, ignore_index=True)

1 loop, best of 3: 1.39 s per loop


In [30]:
%%timeit
d = pd.DataFrame(columns=['A'], index=xrange(1000))
for i in xrange(1000):
    d.loc[i,'A'] = i

1 loop, best of 3: 150 ms per loop


In [31]:
1.39/.150

9.266666666666666

Speedup of nearly 1 order of magnitude

# Revise the code

Go on. Do it.

# Review code changes

In [15]:
%%bash
git log --since 2016-11-07 --oneline | head -n 8

acdd1d2 Finish factoring append out of all operations
a071b77 Add notebook for first iteration of optimizing
78ff930 Start factoring append out of simulation loop
e0c1fbc Remove a concat operation
fe82864 Update tests to use allclose
f13d6c6 fixups
f90e48a Preallocate pl data frame for basal area
d7de2f3 Preallocate dataframes for bafromzero aw, sb, sw


In [16]:
! git diff HEAD~7 ../gypsy

diff --git a/gypsy/GYPSYNonSpatial.py b/gypsy/GYPSYNonSpatial.py
index 35f67d3..24ec050 100644
--- a/gypsy/GYPSYNonSpatial.py
+++ b/gypsy/GYPSYNonSpatial.py
@@ -984,7 +984,7 @@ def BAfromZeroToDataAw(startTage, SI_bh_Aw, N0_Aw, BA_Aw0, SDF_Aw0, f_Aw,
     basal_area_aw_df = pd.DataFrame(columns=['BA_Aw'], index=xrange(max_age))
     BA_tempAw = BA_Aw0
 
-    for SC_Dict, i in enumerate(densities[0: max_age]):
+    for i, SC_Dict in enumerate(densities[0: max_age]):
         bhage_Aw = SC_Dict['bhage_Aw']
         SC_Aw = SC_Dict['SC_Aw']
         N_bh_AwT = SC_Dict['N_bh_AwT']
@@ -1132,7 +1132,7 @@ def BAfromZeroToDataSb(startTage, startTageSb, y2bh_Sb, SC_Sb, SI_bh_Sb,
         t += 1
         startTageSb += 1
 
-    return BA_SbB, BA_Sb_DF
+    return BA_SbB, basal_area_sb_df
 
 
 def BAfactorFinder_Sw(**kwargs):
@@ -1263,7 +1263,7 @@ def BAfromZeroToDataSw(startTage, startTageSw, y2bh_Sw, SC_Sw, SI_bh_Sw,
         t += 1
         startTageSw += 1
 
-    return BA_SwB, BA_Sw_DF
+    

# Tests

There are some issues with the tests - the data does not match the old output data to within 3 or even 2 decimal places. The mismatch is always:

`(mismatch 0.221052631579%)`

It was resolved in fe82864:


In [6]:
%%bash 
git log --since '2016-11-08' --oneline | grep tests

fe82864 Update tests to use allclose


# Run profiling

In [18]:
from gypsy.forward_simulation import simulate_forwards_df

/opt/gypsy/venv/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [19]:
data = pd.read_csv('../private-data/prepped_random_sample_300.csv', index_col=0, nrows=10)

In [20]:
%%prun -D forward-sim-1.prof -T forward-sim-1.txt -q
result = simulate_forwards_df(data)

 
*** Profile stats marshalled to file u'forward-sim-1.prof'. 

*** Profile printout saved to text file u'forward-sim-1.txt'. 


In [21]:
!head forward-sim-1.txt

         10055657 function calls (9875729 primitive calls) in 76.264 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   492069    6.857    0.000    6.857    0.000 GYPSYNonSpatial.py:427(BasalAreaIncrementNonSpatialAw)
  1836602    6.527    0.000    9.190    0.000 {isinstance}
796652/624746    3.102    0.000    4.823    0.000 {len}
     7191    2.670    0.000   40.459    0.006 GYPSYNonSpatial.py:959(BAfromZeroToDataAw)
   511948    2.020    0.000    3.373    0.000 {getattr}


In [22]:
!diff -y forward-sim-1.txt forward-sim.txt

diff: forward-sim.txt: No such file or directory


# Compare performance visualizations

Now use either of these commands to visualize the profiling

```
pyprof2calltree -k -i forward-sim-1.prof forward-sim-1.txt

# or

dc run --service-ports snakeviz notebooks/forward-sim-1.prof
```

### Old

![definitive reference profile screenshot](forward-sim-performance.png)

### New

![1st iteration performance](forward-sim-1-performance.png)

## Summary of performance improvements

forward_simulation is now 4x faster due to the changes outlined in the code review section above

on my hardware, this takes 1000 plots to ~8 minutes

on carol's hardware, this takes 1000 plots to ~25 minutes

For 1 million plots, we're looking at 5 to 17 days on desktop hardware



## Caveat

- this isn't dealing with i/o. reading the plot table in is not a huge problem, especially if we declare the field types, but writing the growth curves for each plot will be time consuming. threads may be necessary

# Identify new areas to optimize



- need to find another order of magnitude improvement to get to 2.4-15 hours 
  - pandas indexing `.ix` (get and set item) is taking 6 and 19% respectively
  - collectively, the lambdas being applied to output data frame are taking 19%
  - BAFromZeroToDataAw is slow (50% of total time) because of (in order):
    - pandas init (dict)
    - baincrementnonspatial
    - pandas setting
    
- parallel (3 cores) gets us to 2 - 6 days - save for last
- AWS with 36 cores gets us to 4 - 12 hours ($6.70 - $20.10 USD on a c4.8xlarge instance in US West Region)

In [27]:
!cat forward-sim-1.txt | grep -i fromzero

     7191    2.670    0.000   40.459    0.006 GYPSYNonSpatial.py:959(BAfromZeroToDataAw)
      444    0.172    0.000    4.115    0.009 GYPSYNonSpatial.py:1207(BAfromZeroToDataSw)
      207    0.087    0.000    2.845    0.014 GYPSYNonSpatial.py:1340(BAfromZeroToDataPl)
       10    0.012    0.001    1.711    0.171 GYPSYNonSpatial.py:1077(BAfromZeroToDataSb)


# Identify some means of optimization

In order of priority/time taken

1. pandas init dict
    - `basal_area_aw_df = pd.DataFrame(columns=['BA_Aw'], index=xrange(max_age))`
    - find a faster way to create this data frame
    - relax the tolerance for aspen
2. pandas set item
    - use at method 
    - http://pandas.pydata.org/pandas-docs/stable/indexing.html#fast-scalar-value-getting-and-setting
3. lambdas
    - use cython for the gross tot vol and merch vol functions
    - might be wise to refactor these first to have conventional names, keyword arguments, and a base implementation to get rid of the boilerplate
    - don't be deceived - the callable is a miniscule portion; series.__getitem__ is taking most of the time
    - again, using .at here would probably be a significant improvement
4. basalareaincremementnonspatialaw
    - this is actually slow because of the number of times the BAFromZeroToDataAw function is called as shown above
    - relaxing the tolerance may help
    - indeed the tolerance is 0.01 * some value while the other factor finder functions have 0.1 tolerance i think
    - can also use cython for the increment functions

do a profiling run with IO (of reading input data and writing the plot curves to files) in next run
